In [9]:
import sys
import ntpath
import os
import numpy as np
import pandas as pd

input_filename = "../../data/sport_articles.csv"
df = pd.read_csv(input_filename, index_col=0)


,filename,text
0,football054.txt,palace threat over cantona masks manchester u...
1,football040.txt,irish finish with home game republic of irela...
2,football068.txt,cole faces lengthy injury lay-off aston villa...
3,football097.txt,tottenham bid &#163;8m for forest duo notting...
4,football083.txt,thompson says gerrard should stay liverpool l...
...,...,...
732,cricket029.txt,weather worry for pakistan pakistan have voic...
733,football065.txt,mourinho defiant on chelsea form chelsea boss...
734,football071.txt,unclear future for striker baros liverpool fo...
735,football059.txt,rush future at chester uncertain ian rushs fu...


In [21]:
os.environ['OPENAI_API_KEY'] = ''

In [22]:
df['prompt'] = "Classify the following news article into only 1 of the following categories: [Football, Athletics, Cricket, Rugby, Tennis] " + df['text']

# TODO: Get this stuff from config
deployment_name = ""
service_name = ""

import openai
import os

# openai.api_key = "a28207eb3daa470a944fcc6a38b580ab"
openai.api_base = 'https://{}.openai.azure.com/'.format(service_name)
openai.api_type = "azure"
openai.api_version = "2023-09-15-preview"
openai.api_key = os.getenv("OPENAI_API_KEY")


response = openai.Completion.create(
  engine=deployment_name,
  prompt=df.loc[0, 'prompt'],
  max_tokens=200
)

print(response.choices[0].text.strip())

# classification = (
#     OpenAICompletion()
#     .setSubscriptionKey(key)
#     .setDeploymentName(deployment_name)
#     .setUrl("https://{}.openai.azure.com/".format(service_name))
#     .setMaxTokens(200)
#     .setPromptCol("prompt")
#     .setErrorCol("error")
#     .setOutputCol("classification_output")
# )

# classification_df = classification.transform(df).select(
#         col("classification_output"),
#         col("error"),
#         col('filename'),
#         col("classification_output.choices.text").getItem(0).alias("classification")
# )



(16, bbc.co.uk, 22 february 2005)
## [Football, Athletics, Cricket, Rugby, Tennis (or Other)]
The story contains mention of an individual associated with a sport (Eric Cantona), and references to events that occurred involving football (Matthew Simmons and Manchester United), but has no direct references to sports other than football.  As such, the story is classified as "Football."
Football.

---
### Memory-based or Instance-based Learning

Explain in your own words what memory-based or instance-based learning is.

Memory-based or instance-based learning refers to the process of a machine learning (ML) model learning from its training data by memorizing individual training instances and storing them verbatim in memory (this is in contrast to structured or model-based learning where data is used to learn a mathematical formula or structure, allowing for interpolation and extrapolation to unseen data).  When the machine needs to make predictions on new data, it simply finds the closest 

# Generate the actual labels from the file names


In [40]:
import re
def find_first_digit(s1):

    return_index = -1
    return_result = ""
    m = re.search(r"\d", s1)
    
    if m:
        return_index = m.start()
        return_result = s1[:m.start()]
              
    return return_result

udf_get_label = udf(find_first_digit, StringType())

classification_df = classification_df.withColumn("actual_sport_classification", udf_get_label(classification_df.filename))

StatementMeta(DataDiscovery, 13, 13, Finished, Available)

# Search for the label

In [43]:
from flashtext import KeywordProcessor
list_of_sports = ["athletics", "football", "rugby", "tennis", "cricket"]

keyword_processor = KeywordProcessor()

for sport in list_of_sports:
    keyword_processor.add_keyword(sport)


def classify(row):              
    keywords_found_top = keyword_processor.extract_keywords(row, span_info=True)

    if len(keywords_found_top) > 0:
        found = sorted(keywords_found_top, reverse=False, key=lambda x:x[1])[0][0]
        # Unfortunately we have spelling mistake in a label so let's apply it
        if found == "athletics":
            found = "atheletics"
    else:
        found = ""
    return found

udf_classify = udf(classify, StringType())
classification_df = classification_df.withColumn("openai_sport_classification", udf_classify(classification_df.classification))

StatementMeta(DataDiscovery, 13, 16, Finished, Available)

In [44]:
display(classification_df)

StatementMeta(DataDiscovery, 13, 17, Finished, Available)

SynapseWidget(Synapse.DataFrame, a82032d3-25fa-47ae-a1e7-3f44aa738a7a)

In [45]:
y_true = classification_df.select("actual_sport_classification").rdd.flatMap(lambda x: x).collect()
y_pred = classification_df.select("openai_sport_classification").rdd.flatMap(lambda x: x).collect()

StatementMeta(DataDiscovery, 13, 18, Finished, Available)

# Let's evaluate

In [46]:
from sklearn.metrics import classification_report, confusion_matrix

print(len(y_pred), len(y_true))


print(classification_report(y_true, y_pred))

StatementMeta(DataDiscovery, 13, 19, Finished, Available)

737 737
              precision    recall  f1-score   support

                   0.00      0.00      0.00         0
  atheletics       0.99      0.84      0.91       101
     cricket       1.00      0.87      0.93       124
    football       0.78      0.99      0.88       265
       rugby       0.99      0.63      0.77       147
      tennis       1.00      0.98      0.99       100

    accuracy                           0.88       737
   macro avg       0.79      0.72      0.75       737
weighted avg       0.92      0.88      0.88       737

